# Test for ForrestAtRsik to use Google Earth Engine using Xee 

This is a simple test to see if ForrestAtRisk can use Google Earth Engine data 1) without credentials (no authentication), 2) using Xee (https://github.com/google/Xee) and the xarray framework, and 3) using `dask` to parallelize the computation.

In [17]:
import ee
import xarray as xr
import rioxarray
from osgeo import ogr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

Google initilization

In [18]:
ee.Initialize()

Area of study: let's go with Peru. Bigger than Martinique, so we can see if the computation is parallelized and how efficient it is

In [19]:
iso3 = "PER"
shp_name = "data_raw/gadm36_" + iso3 + "_0.shp"

Some usefull function

In [20]:
def extent_shp(inShapefile):
    """Compute the extent of a shapefile.

    This function computes the extent (xmin, xmax, ymin, ymax) of a
    shapefile.

    :param inShapefile: Path to the input shapefile.

    :return: The extent as a tuple (xmin, ymin, xmax, ymax).

    """

    inDriver = ogr.GetDriverByName("ESRI Shapefile")
    inDataSource = inDriver.Open(inShapefile, 0)
    inLayer = inDataSource.GetLayer()
    extent = inLayer.GetExtent()
    extent = (extent[0], extent[2], extent[1], extent[3])
    return extent  # (xmin, ymin, xmax, ymax)

Information for task treatment (note that TMF has an updated product up to 2022: https://forobs.jrc.ec.europa.eu/TMF/data#update)

In [21]:
extent_latlong = extent_shp(shp_name)
scale = 0.0002727272727 # 1° ~110km => conversion to original resolution of ~30m
first_year = 1990
last_year = 2022

## New task workflow
This is where the test really starts.

In [22]:
region = ee.Geometry.Rectangle(
    extent_latlong,
    proj="EPSG:4326",
    geodesic=False
    )
region = region.buffer(10000).bounds()
export_coord = region.getInfo()["coordinates"]

In [23]:
AP = ee.ImageCollection("projects/JRC/TMF/v1_2022/AnnualChanges")
ds = xr.open_dataset(
    AP,
    engine='ee',
    crs='EPSG:4326',
    scale=scale,
    geometry=region,
    chunks={"lon": 2000, "lat": 2000}
    ).isel(time=2).squeeze()
ds

/Users/arsouze/miniconda3/envs/forestatrisk/lib/python3.9/site-packages/xee/ext.py:623: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(
/Users/arsouze/miniconda3/envs/forestatrisk/lib/python3.9/site-packages/xarray/core/dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lon" starting at index 2000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/Users/arsouze/miniconda3/envs/forestatrisk/lib/python3.9/site-packages/xarray/core/dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "lat" starting at index 2000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.Dataset> Size: 422GB
Dimensions:  (lon: 47180, lat: 67815)
Coordinates:
    time     int64 8B 2
  * lon      (lon) float32 189kB -81.43 -81.43 -81.42 ... -68.56 -68.56 -68.56
  * lat      (lat) float32 271kB 0.05271 0.05243 0.05216 ... -18.44 -18.44
Data variables: (12/33)
    Dec1990  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec1991  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec1992  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec1993  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec1994  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec1995  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    ...       ...
    Dec2017  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec2018  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec2019  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec2020  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec2021  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    Dec2022  (lon, lat) int32 13GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
Attributes:
    crs:      EPSG:4326

This has been done with no call to gee credentials.

There is a dummy (to be checked) "time" dimension of size 3. For all years, values on dimensions 0 and 1 are all equal to 0, o I don't keep them.

Ok, now that we have lazy access to the data, we have 2 options to proceed:

### Netcdf oriented

We actually reorganize the data to make it more simple to use. Lazy loading.

In [24]:
date_list = range(first_year, last_year + 1)
AP = xr.concat(
    [ds[f"Dec{i}"] for i in date_list],
    dim=pd.Index(date_list, name="years"),
).rename("Annuals Products")
AP

<xarray.DataArray 'Annuals Products' (years: 33, lon: 47180, lat: 67815)> Size: 422GB
dask.array<concatenate, shape=(33, 47180, 67815), dtype=int32, chunksize=(1, 2000, 2000), chunktype=numpy.ndarray>
Coordinates:
    time     int64 8B 2
  * lon      (lon) float32 189kB -81.43 -81.43 -81.42 ... -68.56 -68.56 -68.56
  * lat      (lat) float32 271kB 0.05271 0.05243 0.05216 ... -18.44 -18.44
  * years    (years) int64 264B 1990 1991 1992 1993 1994 ... 2019 2020 2021 2022
Attributes:
    id:             Dec1990
    data_type:      {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max'...
    dimensions:     [282010, 187955]
    crs:            EPSG:4326
    crs_transform:  [0.00026949458523585647, 0, -18.000082337073326, 0, -0.00...

However, the next step of of masking the data becomes much faster.

In [25]:
def forest_masking(AP):
    return xr.where((AP==1) + (AP==2), 1, 0)
AP_forest = AP.map_blocks(forest_masking)
AP_forest

<xarray.DataArray 'Annuals Products' (years: 33, lon: 47180, lat: 67815)> Size: 845GB
dask.array<<this-array>-forest_masking, shape=(33, 47180, 67815), dtype=int64, chunksize=(1, 2000, 2000), chunktype=numpy.ndarray>
Coordinates:
    time     int64 8B dask.array<chunksize=(), meta=np.ndarray>
  * lon      (lon) float32 189kB -81.43 -81.43 -81.42 ... -68.56 -68.56 -68.56
  * years    (years) int64 264B 1990 1991 1992 1993 1994 ... 2019 2020 2021 2022
  * lat      (lat) float32 271kB 0.05271 0.05243 0.05216 ... -18.44 -18.44

We can now compute the forest cover for different desired years.

In [26]:
ref_year = 2020
years_list = (2000, 2005, 2010, 2015, 2020)
def determine_forest(AP_forest, year, ref_year):
    return AP_forest.\
        sel(years=slice(year-1, ref_year+1)).\
        sum("years").\
        rename("forest"+str(year)) >= 1
forest = xr.merge(
    [AP_forest.drop("time").\
    map_blocks(
        determine_forest,
        kwargs={"year":year, "ref_year":ref_year}
        ) for year in years_list]
    )
forest

/var/folders/v1/6y176c8d2jz06qr8rd3jq_480000gn/T/ipykernel_21358/3022250974.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  [AP_forest.drop("time").\
/var/folders/v1/6y176c8d2jz06qr8rd3jq_480000gn/T/ipykernel_21358/3022250974.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  [AP_forest.drop("time").\
/var/folders/v1/6y176c8d2jz06qr8rd3jq_480000gn/T/ipykernel_21358/3022250974.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  [AP_forest.drop("time").\
/var/folders/v1/6y176c8d2jz06qr8rd3jq_480000gn/T/ipykernel_21358/3022250974.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  [AP_forest.drop("time").\
/var/folders/v1/6y176c8d2jz06qr8rd3jq_480000gn/T/ipykernel_21358/3022250974.py:9: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  [AP_forest.drop("time").\


<xarray.Dataset> Size: 16GB
Dimensions:     (lat: 67815, lon: 47180)
Coordinates:
  * lat         (lat) float32 271kB 0.05271 0.05243 0.05216 ... -18.44 -18.44
  * lon         (lon) float32 189kB -81.43 -81.43 -81.42 ... -68.56 -68.56
Data variables:
    forest2000  (lon, lat) bool 3GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    forest2005  (lon, lat) bool 3GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    forest2010  (lon, lat) bool 3GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    forest2015  (lon, lat) bool 3GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    forest2020  (lon, lat) bool 3GB dask.array<chunksize=(2000, 2000), meta=np.ndarray>

Now let's save this file to disk.

In [29]:
from dask.diagnostics import ProgressBar
from zarr import Blosc

compressor = Blosc(cname="zlib", clevel=4)

enc = {}

for k in forest.data_vars:
    enc[k] = {"compressor": compressor}

zarr_delayed = forest.\
                to_zarr(
                    f"forest_{iso3}_compressed.zarr",
                    encoding=enc,
                    compute=False)

with ProgressBar():
    results = zarr_delayed.compute()

[########################################] | 100% Completed | 15m 55s
